# Curso: Bioestatística — DBC (Delineamento em Bloco Casualizado)

## Autores: Sandro da Silva Camargo e Fernando Cardoso

Problema: Suponha que um pesquisador deseja verificar se a potência e o
tempo de microondas (Tratamentos) produzem diferentes resultados para
população de bactérias psicrotroficas (ufc/cm$^2$) (Variável Resposta Y), obtidas de amostras (50cm$^2$) (UE) de carcaça de frangos resfriados.

Tratamentos:
*  700 w e 2 min.;
*  350 w e 1 min.; e
*  Controle.

O pesquisador decidiu usar seis repetições por tratamento e fazer as
medições ao longo de seis dias, desse modo, as repetições (blocos)
são os dias.

Como as UE provavelmente comportam-se de modo diferente nestes
dias (mais calor, menos calor, etc.), isto pode inflacionar o erro
experimental.

Assim, deseja-se remover a variabilidade entre unidades do erro
experimental. Para este fim, será usado cada tratamento apenas
uma vez em cada um dos 6 dias. Dentro do bloco (dias), a ordem
de aplicação dos tratamentos deve ser realizada de forma aleatória.

A base de dados está disponível [aqui](https://raw.githubusercontent.com/Sandrocamargo/biostatistics/refs/heads/master/datasets/dbc-dados.txt).

Abra este código no seu google colab clicando [aqui](http://colab.research.google.com/github/Sandrocamargo/biostatistics/blob/master/python/bioe_04_dbc.ipynb).

In [ ]:
# %% Cleaning Environment
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.stats.stattools import durbin_watson
from scipy.stats import levene
!mkdir images

In [ ]:
# %% Load data
data = pd.read_csv("https://raw.githubusercontent.com/Sandrocamargo/biostatistics/refs/heads/master/datasets/dbc-dados.txt", decimal=",", sep=" ", header=0)
data.columns = ['Treatment', 'Block', 'Y']
data['Treatment'] = data['Treatment'].astype('category')
data['Block'] = data['Block'].astype('category')

# O grupo de controle é o primeiro desta lista
data["Treatment"] = pd.Categorical(
    data["Treatment"],
    categories=["Controle", "350w1m", "700w2m"],
    ordered=True
)

data.info()
data.describe()

In [ ]:
# %% AOV model
from patsy.contrasts import Treatment
model = ols('Y ~ C(Treatment) + C(Block)', data=data).fit()
print(model.summary())

# Ajuste do modelo (OLS)

*  Variável dependente: Y
*  Modelo: Regressão linear com fatores Treatment e Block.
*  R² = 0.900 → O modelo explica 90% da variação total em Y, o que indica um ajuste muito bom.
*  R² ajustado = 0.830 → Considerando o número de variáveis, ainda explica 83% da variação, mantendo robustez.
*  F-statistic = 12.85, p = 0.000284 → O modelo como um todo é estatisticamente significativo; pelo menos uma variável explicativa contribui para explicar Y.

# Coeficientes do modelo
### Resultados da Regressão Linear (OLS)

| Aspecto | Valor | Interpretação |
|:--|:--:|:--|
| **R²** | 0.900 | O modelo explica 90% da variação em Y. |
| **R² ajustado** | 0.830 | Bom ajuste considerando o número de variáveis. |
| **F do modelo** | 12.85 | O modelo é globalmente significativo. |
| **p(F)** | 0.000284 | Há evidência de efeito global. |
| **Tratamento (ANOVA)** | F = 27.90, p = 0.000081 | O tratamento afeta significativamente Y. |
| **Bloco (ANOVA)** | F = 6.83, p = 0.0051 | O bloco também tem efeito significativo. |
| **Intercepto** | 13.5933 (p < 0.001) | Valor médio de Y no grupo de referência (350w1m, Bloco I). |
| **Tratamento 350w1m** | 1.0050 (p < 0.001) | Significativamente maior que o grupo de Controle. |
| **Tratamento 700w2m** | 1.0550 (p < 0.001) | Significativamente maior que o grupo de Controle. |
| **Bloco II** | 0.590 (p = 0.026) | Valor significativamente maior que o Bloco I. |
| **Bloco III** | -0.463 (p = 0.067) | Tendência a ser menor que o Bloco I (não significativo a 5%). |
| **Bloco IV** | 0.243 (p = 0.306) | Diferença não significativa. |
| **Bloco V** | 0.537 (p = 0.039) | Valor significativamente maior que o Bloco I. |
| **Bloco VI** | 0.563 (p = 0.031) | Valor significativamente maior que o Bloco I. |
| **Durbin–Watson** | 2.024 | Resíduos independentes (sem autocorrelação). |
| **Normalidade dos resíduos (JB)** | p = 0.677 | Resíduos aproximadamente normais. |

# Resumo
*  O grupo Controle apresenta menor Y que os grupos experimentais.
*  Alguns blocos têm efeito significativo sobre Y (BlocoII, BlocoV, BlocoVI).
*  O grupo 700w2m não difere do grupo 350w1m.

In [ ]:
anova_results = anova_lm(model)
print("\n=== ANOVA ===")
print(anova_results)

# %% Compute totals
summary_table = pd.DataFrame({
    'Length': data.groupby('Treatment', observed=False)['Y'].count(),
    'Sum': data.groupby('Treatment', observed=False)['Y'].sum(),
    'Sum_Squares': data.groupby('Treatment', observed=False)['Y'].apply(lambda x: np.sum(x**2)),
    'Mean': data.groupby('Treatment', observed=False)['Y'].mean()
})
print("\n=== Summary by Treatment ===")
print(summary_table)

# ANOVA do modelo

|Fonte | F| p-valor	| Interpretação|
|:--|:--:|:--|:--|
|Treatment | 27.90	| 0.000081	| Efeito do tratamento altamente significativo sobre Y. |
| Block | 6.83 | 0.005 | Efeito do bloco também significativo. |
| Residual | – | – | Variabilidade não explicada. |

# Estatísticas descritivas por tratamento

| Treatment | Média (mean) | Interpretação|
|:--|:--:|:--|
|Controle|13.84|Menor média → diferença significativa com os grupos experimentais|
|350w1m|14.84|Grupo de referência|
|700w2m|14.89|Sem diferença relevante → não significativo |

# Diagnóstico do modelo
*  Durbin-Watson = 2.024 → resíduos independentes (não há autocorrelação).
*  Omnibus p = 0.487, Jarque-Bera p = 0.677 → resíduos aproximadamente normais.
*  Cond. No = 7.57 → sem problemas graves de multicolinearidade.

# Conclusões principais
*  O modelo explica bem a variação em Y (R² = 0.9).
*  O Tratamento tem efeito significativo, principalmente o grupo Controle, que é menor que os grupos experimentais.
*  O Bloco também influencia, embora apenas alguns blocos sejam significativamente diferentes do BlocoI.
*  Os resíduos são independentes e normalmente distribuídos → suposições da ANOVA/OLS atendidas.

In [ ]:
# %% Shapiro-Wilk test (normality)
shapiro_test = stats.shapiro(model.resid)
stat, p = shapiro_test.statistic, shapiro_test.pvalue

print("=== Teste de Normalidade de Shapiro-Wilk ===")
print(f"Estatística W = {stat:.4f}")
print(f"Valor-p = {p:.4f}")

if p > 0.05:
    print("✅ Não há evidências para rejeitar a normalidade dos resíduos (p > 0.05).")
else:
    print("⚠️ Evidências de que os resíduos não seguem distribuição normal (p ≤ 0.05).")

plt.hist(model.resid, bins=10, edgecolor='black')
plt.xlabel("Errors")
plt.ylabel("Frequency")
plt.title("Errors Histogram")
plt.savefig('images/dbc-errorshistogram.pdf')
plt.show()

# %% Bartlett tests (homogeneidade de variâncias)
bartlett_treatment = stats.bartlett(*[group["Y"].values for name, group in data.groupby("Treatment", observed=False)])
bartlett_block = stats.bartlett(*[group["Y"].values for name, group in data.groupby("Block", observed=False)])

print("=== Teste de Bartlett: Homogeneidade de variâncias ===\n")

print("Por Tratamento:")
print(f"  Estatística de Bartlett = {bartlett_treatment.statistic:.4f}")
print(f"  Valor-p = {bartlett_treatment.pvalue:.4f}")
if bartlett_treatment.pvalue > 0.05:
    print("✅ As variâncias entre os tratamentos são homogêneas (p > 0,05).")
else:
    print("⚠️As variâncias entre os tratamentos são diferentes (p ≤ 0,05).")

print("\nPor Bloco:")
print(f"  Estatística de Bartlett = {bartlett_block.statistic:.4f}")
print(f"  Valor-p = {bartlett_block.pvalue:.4f}")
if bartlett_block.pvalue > 0.05:
    print("✅ As variâncias entre os blocos são homogêneas (p > 0,05).")
else:
    print("⚠️As variâncias entre os blocos são diferentes (p ≤ 0,05).")


# Boxplots
plt.boxplot([model.resid[data['Treatment'] == t] for t in data['Treatment'].cat.categories])
plt.xlabel("Treatments")
plt.ylabel("Errors")
plt.title("Homogeneity of Errors (Treatment)")
plt.savefig('images/dbc-homogeneityErrors-tr.pdf')
plt.show()

plt.boxplot([model.resid[data['Block'] == b] for b in data['Block'].cat.categories])
plt.xlabel("Blocks")
plt.ylabel("Errors")
plt.title("Homogeneity of Errors (Block)")
plt.savefig('images/dbc-homogeneityErrors-bl.pdf')
plt.show()

# %% Levene test (alternative for homogeneity)
levene_treatment = stats.levene(*[group["Y"].values for name, group in data.groupby("Treatment", observed=False)])
levene_block = stats.levene(*[group["Y"].values for name, group in data.groupby("Block", observed=False)])

print("=== Teste de Levene: Homogeneidade de variâncias ===\n")

print("Por Tratamento:")
print(f"  Estatística de Levene = {levene_treatment.statistic:.4f}")
print(f"  Valor-p = {levene_treatment.pvalue:.4f}")
if levene_treatment.pvalue > 0.05:
    print("✅ As variâncias entre os tratamentos são homogêneas (p > 0,05).")
else:
    print("⚠️ As variâncias entre os tratamentos são diferentes (p ≤ 0,05).")

print("\nPor Bloco:")
print(f"  Estatística de Levene = {levene_block.statistic:.4f}")
print(f"  Valor-p = {levene_block.pvalue:.4f}")
if levene_block.pvalue > 0.05:
    print("✅ As variâncias entre os blocos são homogêneas (p > 0,05).")
else:
    print("⚠️ As variâncias entre os blocos são diferentes (p ≤ 0,05).")

# Residuals vs factors
plt.scatter(data['Treatment'].cat.codes, model.resid)
plt.xlabel("Treatments")
plt.ylabel("Errors")
plt.title("Residuals by Treatment")
plt.savefig('images/dbc-levene-tr.pdf')
plt.show()

plt.scatter(data['Block'].cat.codes, model.resid)
plt.xlabel("Blocks")
plt.ylabel("Errors")
plt.title("Residuals by Block")
plt.savefig('images/dbc-levene-bl.pdf')
plt.show()

# %% Durbin-Watson test (independence)
print("=== Teste de Durbin–Watson: Independência dos resíduos ===\n")
dw = durbin_watson(model.resid)
print(f"Estatística de Durbin–Watson = {dw:.3f}")

# Interpretação aproximada
if dw < 1.5:
    interpret = "⚠️ Indica possível autocorrelação positiva nos resíduos."
elif dw > 2.5:
    interpret = "⚠️ Indica possível autocorrelação negativa nos resíduos."
else:
    interpret = "✅ Não há evidências de autocorrelação — resíduos independentes."

print(interpret)


plt.scatter(range(len(model.resid)), model.resid, color='red')
plt.xlabel("Samples")
plt.ylabel("Errors")
plt.title("Errors Independence")
plt.savefig('images/dbc-errorsindependence.pdf')
plt.show()

# %% Q-Q plot (normality check)
sm.qqplot(model.resid, line='45', fit=True)
plt.title("Q-Q Plot of Residuals")
plt.savefig('images/dbc-qqplot.pdf')
plt.show()

# Os pressupostos foram atendidos:
*  ✅ Normalidade dos resíduos;
*  ✅ Homogeneidade de variâncias; e
*  ✅ Independência dos erros.

In [ ]:
# %% Tukey HSD test (multiple comparison)
print("\n=== Tukey HSD Test (Treatment) ===")
tukey_tr = pairwise_tukeyhsd(data['Y'], data['Treatment'])
print(tukey_tr.summary())

# Gráfico de médias (Tukey)
means = data.groupby('Treatment', observed=False)['Y'].mean()
means.plot(kind='bar', color='skyblue')
plt.title("Tukey Test - Treatments")
plt.xlabel("Treatments")
plt.ylabel("Means")
plt.ylim(0, data['Y'].max() * 1.1)
plt.savefig("images/dbc-tukey-tr-aula.pdf")
plt.show()

print("\n=== Tukey HSD Test (Block) ===")
tukey_bl = pairwise_tukeyhsd(data['Y'], data['Block'])
print(tukey_bl.summary())

# %% Fitted model diagnostic plots
sm.graphics.plot_regress_exog(model, "C(Treatment)[T.2]" if "C(Treatment)[T.2]" in model.params else model.params.index[1])
plt.show()

plt.scatter(model.fittedvalues, model.resid)
plt.xlabel("Predicted")
plt.ylabel("Errors")
plt.title("Predicted vs Errors")
plt.savefig('images/dbc-lm-pred.pdf')
plt.show()

# %% Means and variances per Block
tot = pd.DataFrame({
    'Means': data.groupby('Block', observed=False)['Y'].mean(),
    'Variances': data.groupby('Block', observed=False)['Y'].var()
})
print("\n=== Means and Variances per Block ===")
print(tot)


# Conclusões

Tukey HSD — Treatment
|Comparação|Diferença de Médias|p-ajustado|IC 95% Inferior|IC 95% Superior	|Significativo|
|:--|:--:|:--:|:--:|:--:|:--:|
|350w1m vs 700w2m|	0.050|	0.9818|	-0.6601|	0.7601|	❌
|350w1m vs Controle|	-1.005|	0.0060|	-1.7151|	-0.2949|	✅
|700w2m vs Controle|	-1.055|	0.0041|	-1.7651|	-0.3449|	✅

🟢 **Conclusão**: Ambos os tratamentos (350w1m e 700w2m) diferem significativamente do controle, mas não diferem entre si.

Tukey HSD — Block
|Comparação|Diferença de Médias|p-ajustado|IC 95% Inferior|IC 95% Superior	|Significativo|
|:--|:--:|:--:|:--:|:--:|:--:|
|BlocoI vs BlocoII	|0.5900|	0.8649|	-1.1826|	2.3626|	❌
|BlocoI vs BlocoIII	|-0.4633|	0.9449|	-2.2360|	1.3093|	❌
|BlocoI vs BlocoIV	|0.2433|	0.9967|	-1.5293|	2.0160|	❌
|BlocoI vs BlocoV	|0.5367|	0.9035|	-1.2360|	2.3093|	❌
|BlocoI vs BlocoVI	|0.5633|	0.8851|	-1.2093|	2.3360|	❌
|BlocoII vs BlocoIII	|-1.0533|	0.3975|	-2.8260|	0.7193|	❌
|BlocoII vs BlocoIV	|-0.3467|	0.9836|	-2.1193|	1.4260|	❌
|BlocoII vs BlocoV	|-0.0533|	1.0000|	-1.8260|	1.7193|	❌
|BlocoII vs BlocoVI	|-0.0267|	1.0000|	-1.7993|	1.7460|	❌
|BlocoIII vs BlocoIV	|0.7067|	0.7596|	-1.0660|	2.4793|	❌
|BlocoIII vs BlocoV	|1.0000|	0.4490|	-0.7726|	2.7726|	❌
|BlocoIII vs BlocoVI	|1.0267|	0.4228|	-0.7460|	2.7993|	❌
|BlocoIV vs BlocoV	|0.2933|	0.9922|	-1.4793|	2.0660|	❌
|BlocoIV vs BlocoVI	|0.3200|	0.9885|	-1.4526|	2.0926|	❌
|BlocoV vs BlocoVI	|0.0267|	1.0000|	-1.7460|	1.7993|	❌

🔵 **Conclusão**: Nenhuma diferença significativa entre os blocos.